# 08 - Portfolio with Option Overlay

Capstone: compare unhedged underlying exposure against a budget-constrained protective put overlay.

In [ ]:
from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / 'src').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import pandas as pd

from src.config import config_dict
from src.portfolio_overlay import protective_put_overlay_simulation, overlay_risk_metrics
from src.plotting import plot_overlay_distribution

cfg = config_dict(fast_mode=True)


In [ ]:
overlay = protective_put_overlay_simulation(
    S0=cfg['S0'],
    r=cfg['R'],
    q=cfg['Q'],
    sigma=cfg['SIGMA'],
    T_sim=cfg['T'],
    K_put=cfg['PROTECTIVE_PUT_STRIKE'],
    T_put=cfg['PUT_MATURITY'],
    notional=cfg['PORTFOLIO_NOTIONAL'],
    premium_budget_fraction=cfg['HEDGE_BUDGET_FRACTION'],
    n_paths=40_000,
    n_steps=252,
    seed=cfg['SEED'],
)

metrics = overlay_risk_metrics(overlay['unhedged_returns'], overlay['hedged_returns'])
metrics['premium_paid'] = [overlay['premium_paid'], overlay['premium_paid']]
metrics['coverage_ratio'] = [overlay['coverage_ratio'], overlay['coverage_ratio']]
metrics.to_csv(ROOT / 'results' / 'tables' / 'overlay_metrics.csv')

plot_overlay_distribution(
    overlay['unhedged_returns'],
    overlay['hedged_returns'],
    ROOT / 'results' / 'figures' / 'overlay_return_dist.png',
)

metrics
